In [15]:
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
df=pd.read_csv("/content/drive/MyDrive/1. Clasificacion/Muestras/muestra_unida_genero.csv", encoding="utf-8")
df.head()

,Item,Marca,Precio,Categoria,Web,class_inei,codigo,genero
0,sandalia mujer felicina,hush puppies,295.20,sandalias mujeres,falabella,SANDALIAS (MUJER > 14),321030115,mujer
1,abrigo acolchado para mujer,blwoens,199.00,moda mujer,falabella,"CASACA, CHAQUETA, GUERRERA (MUJER > 14)",312020114,mujer
2,camisa lino manga larga hombre,isague,84.00,moda hombre,falabella,CAMISA (HOMBRE > 14),312010124,hombre
3,gorro algodón hombre,polo ralph lauren,229.90,gorros y sombreros,falabella,NO CLASIFICACIÓN,0,hombre
4,top lila mujer top corto,urpi dreams,39.98,moda mujer,falabella,"POLO (INCLUYE TIPO BIBIDI, TOPS, ETC.) (MUJER ...",312020142,mujer


In [ ]:
df.shape

(5683, 8)

## Datos nulos

In [ ]:
df.isnull().sum()

,0
Item,0
Marca,0
Precio,0
Categoria,0
Web,0
class_inei,0
codigo,0
genero,0


## Pre-Procesamiento

In [ ]:
#Conviertiendo a minúsculas
df["Item"]=df["Item"].str.lower()

#Eliminando espacios en blaco al comienzo y al final
df["Item"]=df["Item"].str.strip()

#Eliminando ": ; ,"
df["Item"]=df["Item"].str.replace(":","")
df["Item"]=df["Item"].str.replace(";","")
df["Item"]=df["Item"].str.replace(",","")

#Eliminar espacios redundantes entre palabras
df["Item"] = df["Item"].str.replace(r'\s+', ' ', regex=True)


df.head()

,Item,Marca,Precio,Categoria,Web,class_inei,codigo,genero
0,sandalia mujer felicina,hush puppies,295.20,sandalias mujeres,falabella,SANDALIAS (MUJER > 14),321030115,mujer
1,abrigo acolchado para mujer,blwoens,199.00,moda mujer,falabella,"CASACA, CHAQUETA, GUERRERA (MUJER > 14)",312020114,mujer
2,camisa lino manga larga hombre,isague,84.00,moda hombre,falabella,CAMISA (HOMBRE > 14),312010124,hombre
3,gorro algodón hombre,polo ralph lauren,229.90,gorros y sombreros,falabella,NO CLASIFICACIÓN,0,hombre
4,top lila mujer top corto,urpi dreams,39.98,moda mujer,falabella,"POLO (INCLUYE TIPO BIBIDI, TOPS, ETC.) (MUJER ...",312020142,mujer


## Clases de producto

In [ ]:
df["codigo"].unique()

array([321030115, 312020114, 312010124,         0, 312020142, 312010160,
       312030158, 312040142, 321040118, 312010119, 312020137, 321030106,
       321010303, 321030121, 312020172, 321010121, 312050108, 312050138,
       321050106, 312010153, 312030167, 312010138, 321020118, 321030303,
       312010132, 313010115, 321040303, 312020168, 321040121, 312020165,
       312010154, 312020131, 321020303, 321030309, 312020106, 312020117,
       312040137, 312010158, 312010171, 312020118, 312030169, 312040116,
       312020136, 312050152, 312030150, 312040165, 312010167, 312060230,
       312010186, 312040169, 321020307, 321010307, 322010103, 313060103])

In [ ]:
df["class_inei"].unique().shape

(54,)

In [ ]:
df['class_inei'].value_counts()

,count
class_inei,
NO CLASIFICACIÓN,823
ZAPATILLAS (MUJER > 14),455
POLO (HOMBRE > 14),413
ZAPATILLAS (HOMBRE >14),376
SANDALIAS (MUJER > 14),365
PANTALÓN (MUJER > 14),241
ROPA INTERIOR (MUJER > 14),237
"POLO (INCLUYE TIPO BIBIDI, TOPS, ETC.) (MUJER > 14)",215
BOTAS (MUJER > 14),196


In [ ]:
df[df["class_inei"]=="REPARACIÓN DE ZAPATO (HOMBRE > 14)"]

,Item,Marca,Precio,Categoria,Web,class_inei,codigo,genero
5250,zapato mocasin chato paula ii botella,corelli,159.99,mocasines,falabella,REPARACIÓN DE ZAPATO (HOMBRE > 14),322010103,mujer


FASTTEXT-

In [ ]:
!pip install fasttext

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.4/73.4 kB 2.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached pybind11-2.13.6-py3-none-any.whl.metadata (9.5 kB)
Using cached pybind11-2.13.6-py3-none-any.whl (243 kB)
  Created wheel for fasttext: filename=fasttext-0.9.3-cp310-cp310-linux_x86_64.whl size=4296189 sha256=a8ee20cf755a90d252bc85334677f112f7c66755b311934d07afd06492872c57
  Stored in directory: /root/.cache/pip/wheels/0d/a2/00/81db54d3e6a8199b829d58e02cec2ddb20ce3e59fad8d3c92a
Successfully built fasttext


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5683 entries, 0 to 5682
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Item        5683 non-null   object 
 1   Marca       5683 non-null   object 
 2   Precio      5683 non-null   float64
 3   Categoria   5683 non-null   object 
 4   Web         5683 non-null   object 
 5   class_inei  5683 non-null   object 
 6   codigo      5683 non-null   int64  
 7   genero      5683 non-null   object 
dtypes: float64(1), int64(1), object(6)
memory usage: 355.3+ KB


In [ ]:
from sklearn.model_selection import train_test_split

X=df.drop(columns=["class_inei","codigo","Web"])
y=df["codigo"].astype(str)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [16]:
import fasttext

# Ruta para guardar temporalmente los textos
path = "/content/"

#Guarda los nombres en un archivo de texto
product_names = df["Item"]

product_names.to_csv(f"{path}/products.txt", index=False, header=False)

# Parámetros comunes para FastText
params_cbow = {
    "input": f"{path}/products.txt",
    "lr": 0.1,
    "dim": 100,
    "epoch": 20,
    "wordNgrams": 3,
    "minn": 3,
    "maxn": 6,
    "ws": 4,
    "minCount": 1,
    "thread": 4,
}

params_skip = params_cbow.copy()  # Copiamos la configuración base
params_skip.update({"model": "skipgram"})  # Cambiar el modelo a Skip-gram

# Entrenar CBOW y Skip-gram
model_cbow = fasttext.train_unsupervised(**params_cbow, model="cbow")
model_skip = fasttext.train_unsupervised(**params_skip)

# Guardar los modelos manualmente
model_cbow.save_model(f"{path}/cbow_word_vectors.bin")
model_skip.save_model(f"{path}/skip_word_vectors.bin")

# Función para generar embeddings
def build_fasttext_embedding(text, model):
  words = str(text).split()  # Divide en palabras
  return np.mean([model.get_word_vector(w) for w in words if w in model.words], axis=0)
  #promedio de vectores para obtenr un unico embedding



# Aplicar la función para generar representaciones vectoriales con CBOW
train_features_cbow = X_train['Item'].apply(lambda x: build_fasttext_embedding(x, model_cbow))
test_features_cbow = X_test['Item'].apply(lambda x: build_fasttext_embedding(x, model_cbow))

# Aplicar la función para generar representaciones vectoriales con Skip-gram
train_features_skip = X_train['Item'].apply(lambda x: build_fasttext_embedding(x, model_skip))
test_features_skip = X_test['Item'].apply(lambda x: build_fasttext_embedding(x, model_skip))

# Preparar datos para ML
def prepare_ml_data(features, codes):
  features = features.reset_index(drop=True)
  codes = codes.reset_index(drop=True)
  df = pd.DataFrame(features.tolist())  # Convertir embeddings a DataFrame
  df.insert(0, 'cod', codes)  # Insertar códigos como columna
  return df


#Aplicar la función prepare_ml_data para obtener un data frame
# CBOW Features como entradas para el modelo
train_cbow_ml = prepare_ml_data(train_features_cbow, y_train)
test_cbow_ml = prepare_ml_data(test_features_cbow, y_test)

# Skip-gram Features como entradas para el modelo
train_skip_ml = prepare_ml_data(train_features_skip, y_train)
test_skip_ml = prepare_ml_data(test_features_skip, y_test)

# Guardar las características listas para ML
train_cbow_ml.to_csv('train_cbow_ml.csv', index=False)
test_cbow_ml.to_csv('test_cbow_ml.csv', index=False)
train_skip_ml.to_csv('train_skip_ml.csv', index=False)
test_skip_ml.to_csv('test_skip_ml.csv', index=False)

In [17]:
#SVM
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report
import pandas as pd

# Cargar las características generadas (CBOW y Skip-gram)
train_cbow_ml = pd.read_csv('train_cbow_ml.csv')  # Asegúrate de que este archivo exista
test_cbow_ml = pd.read_csv('test_cbow_ml.csv')
train_skip_ml = pd.read_csv('train_skip_ml.csv')
test_skip_ml = pd.read_csv('test_skip_ml.csv')

# Separar características (X) y etiquetas (y) para CBOW
X_train_cbow = train_cbow_ml.drop(columns=['cod'])  # Quitar la columna 'cod'
y_train_cbow = train_cbow_ml['cod']
X_test_cbow = test_cbow_ml.drop(columns=['cod'])
y_test_cbow = test_cbow_ml['cod']

# Separar características (X) y etiquetas (y) para Skip-gram
X_train_skip = train_skip_ml.drop(columns=['cod'])
y_train_skip = train_skip_ml['cod']
X_test_skip = test_skip_ml.drop(columns=['cod'])
y_test_skip = test_skip_ml['cod']

# Entrenamiento de SVM para CBOW
print("Entrenando SVM con CBOW embeddings.")
svm_cbow = SVC(kernel='rbf')
param_grid = {'C': [0.001,0.1, 1, 10,100]}  # Parámetros de búsqueda
grid_search_cbow = GridSearchCV(svm_cbow, param_grid, cv=5, scoring='accuracy')
grid_search_cbow.fit(X_train_cbow, y_train_cbow)

# Mejor modelo CBOW
best_model_cbow = grid_search_cbow.best_estimator_
print(f"Mejor parámetro C para CBOW: {grid_search_cbow.best_params_}")

# Predicciones para CBOW
pred_cbow = best_model_cbow.predict(X_test_cbow)

# Evaluación para CBOW
print("Matriz de confusión para CBOW:")
print(confusion_matrix(y_test_cbow, pred_cbow))
print("Reporte de clasificación para CBOW:")
print(classification_report(y_test_cbow, pred_cbow))

# Entrenamiento de SVM para Skip-gram
print("Entrenando SVM con Skip-gram embeddings")
svm_skip = SVC(kernel='rbf')
grid_search_skip = GridSearchCV(svm_skip, param_grid, cv=5, scoring='accuracy')
grid_search_skip.fit(X_train_skip, y_train_skip)

# Mejor modelo Skip-gram
best_model_skip = grid_search_skip.best_estimator_
print(f"Mejor parámetro C para Skip-gram: {grid_search_skip.best_params_}")

# Predicciones para Skip-gram
pred_skip = best_model_skip.predict(X_test_skip)

# Evaluación para Skip-gram
print("Matriz de confusión para Skip-gram:")
print(confusion_matrix(y_test_skip, pred_skip))
print("Reporte de clasificación para Skip-gram:")
print(classification_report(y_test_skip, pred_skip))

# Guardar las predicciones en un archivo CSV
pd.DataFrame({'Real': y_test_cbow, 'Predicción CBOW': pred_cbow}).to_csv('predicciones_cbow.csv', index=False)
pd.DataFrame({'Real': y_test_skip, 'Predicción Skip-gram': pred_skip}).to_csv('predicciones_skip.csv', index=False)

Entrenando SVM con CBOW embeddings.


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:776: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(


Mejor parámetro C para CBOW: {'C': 100}
Matriz de confusión para CBOW:
[[121   4   0 ...   0   1   0]
 [  5   9   0 ...   0   0   0]
 [  0   0  29 ...   0   0   0]
 ...
 [  2   0   0 ...   1   0   0]
 [  2   0   0 ...   0   9   0]
 [  1   0   0 ...   0   0   0]]
Reporte de clasificación para CBOW:
              precision    recall  f1-score   support

           0       0.47      0.71      0.57       170
   312010119       0.60      0.60      0.60        15
   312010124       0.88      0.94      0.91        31
   312010132       0.65      0.65      0.65        23
   312010138       0.00      0.00      0.00         1
   312010153       1.00      0.43      0.60         7
   312010154       0.83      0.92      0.88        38
   312010158       0.76      0.67      0.71        24
   312010160       0.84      0.90      0.87        90
   312010167       1.00      0.50      0.67         4
   312010171       1.00      0.83      0.91        12
   312020106       0.75      0.75      0.75        2

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/m

Mejor parámetro C para Skip-gram: {'C': 10}
Matriz de confusión para Skip-gram:
[[111   1   1 ...   0   0   0]
 [  3   9   0 ...   0   0   0]
 [  0   0  31 ...   0   0   0]
 ...
 [  0   0   0 ...   2   0   0]
 [  3   0   0 ...   0   9   0]
 [  0   0   0 ...   0   0   1]]
Reporte de clasificación para Skip-gram:
              precision    recall  f1-score   support

           0       0.50      0.65      0.56       170
   312010119       0.75      0.60      0.67        15
   312010124       0.79      1.00      0.89        31
   312010132       0.84      0.70      0.76        23
   312010138       1.00      1.00      1.00         1
   312010153       1.00      0.43      0.60         7
   312010154       0.86      0.97      0.91        38
   312010158       0.76      0.67      0.71        24
   312010160       0.85      0.89      0.87        90
   312010167       0.00      0.00      0.00         4
   312010171       0.90      0.75      0.82        12
   312020106       0.87      0.83     

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


# Data final

## Prediciendo los códigos de la data extraida el 26/11

In [ ]:
# Data final 26/11
df1 = pd.read_csv("/content/drive/MyDrive/1. Clasificacion/Muestras/26_11_24_genero.csv", encoding="utf-8")
df1.head()

,Item,Marca,Precio,Categoria,Web,Fecha,genero
0,zapatillas urbanas adidas para hombre,adidas,160.30,zapatillas,oeschle,26-11-2024,hombre
1,zapatillas urbanas reebok mujer reebok jogger ...,reebok,129.00,zapatillas,oeschle,26-11-2024,mujer
2,zapatillas urbanas reebok para hombre jogg lite,reebok,182.78,zapatillas,oeschle,26-11-2024,hombre
3,zapatillas urbanas reebok hombre jogger lite,reebok,229.00,zapatillas,oeschle,26-11-2024,hombre
4,zapatillas urbanas para mujer adidas court pl...,adidas,249.00,zapatillas,oeschle,26-11-2024,mujer


In [ ]:
df1 = df1.dropna(subset=['Precio'])
df1.isnull().sum()

,0
Item,0
Marca,0
Precio,0
Categoria,0
Web,0
Fecha,0
genero,0


### Pre - Procesamiento data 26/11

In [ ]:
#Conviertiendo a minúsculas
df1["Item"]=df1["Item"].str.lower()

#Eliminando espacios en blaco al comienzo y al final
df1["Item"]=df1["Item"].str.strip()

#Eliminando ": ; ,"
df1["Item"]=df1["Item"].str.replace(":","")
df1["Item"]=df1["Item"].str.replace(";","")
df1["Item"]=df1["Item"].str.replace(",","")

#Eliminar espacios redundantes entre palabras
df1["Item"] = df1["Item"].str.replace(r'\s+', ' ', regex=True)

df1.head()

,Item,Marca,Precio,Categoria,Web,Fecha,genero
0,zapatillas urbanas adidas para hombre,adidas,160.30,zapatillas,oeschle,26-11-2024,hombre
1,zapatillas urbanas reebok mujer reebok jogger ...,reebok,129.00,zapatillas,oeschle,26-11-2024,mujer
2,zapatillas urbanas reebok para hombre jogg lite,reebok,182.78,zapatillas,oeschle,26-11-2024,hombre
3,zapatillas urbanas reebok hombre jogger lite,reebok,229.00,zapatillas,oeschle,26-11-2024,hombre
4,zapatillas urbanas para mujer adidas court pla...,adidas,249.00,zapatillas,oeschle,26-11-2024,mujer


In [ ]:
# Aplicar la función para generar representaciones vectoriales con Skip-gram
x_features_skip = df1['Item'].apply(lambda x: build_fasttext_embedding(x, model_skip))

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [ ]:
nan_count = sum(1 for x in x_features_skip if isinstance(x, float) and np.isnan(x))
print(f"Número de valores NaN: {nan_count}")

Número de valores NaN: 48


In [ ]:
# Identificar los índices donde hay NaN
nan_indices = [i for i, feature in enumerate(x_features_skip) if isinstance(feature, float) and np.isnan(feature)]

print(f"Índices con NaN: {nan_indices}")

Índices con NaN: [6888, 28087, 28089, 36978, 37048, 37076, 37395, 37448, 37470, 37578, 38628, 38846, 39100, 41526, 41691, 42740, 43568, 44327, 44530, 45167, 46010, 46011, 46772, 52453, 52454, 52455, 53392, 53399, 55825, 56542, 57323, 57341, 57385, 57386, 57423, 61240, 62727, 66640, 67729, 67743, 68666, 73632, 74282, 79615, 81394, 81577, 81659, 81808]


In [ ]:
# Eliminar las filas de df2 con esos índices
df1_clean = df1.drop(index=nan_indices)
df1_clean = df1_clean.reset_index(drop=True)
df1_clean

,Item,Marca,Precio,Categoria,Web,Fecha,genero
0,zapatillas urbanas adidas para hombre,adidas,160.30,zapatillas,oeschle,26-11-2024,hombre
1,zapatillas urbanas reebok mujer reebok jogger ...,reebok,129.00,zapatillas,oeschle,26-11-2024,mujer
2,zapatillas urbanas reebok para hombre jogg lite,reebok,182.78,zapatillas,oeschle,26-11-2024,hombre
3,zapatillas urbanas reebok hombre jogger lite,reebok,229.00,zapatillas,oeschle,26-11-2024,hombre
4,zapatillas urbanas para mujer adidas court pla...,adidas,249.00,zapatillas,oeschle,26-11-2024,mujer
...,...,...,...,...,...,...,...
84597,pantalón straight algodón mujer,stefano cocci,129.90,moda mujer,falabella,26-11-2024,mujer
84598,conjunto hombre polera más joggers entero hech...,generico,139.90,moda hombre,falabella,26-11-2024,hombre
84599,pantalón casual mujer,springfield,99.96,moda mujer,falabella,26-11-2024,mujer
84600,pantalón casual mujer,springfield,124.95,moda mujer,falabella,26-11-2024,mujer


In [ ]:
# Aplicar la función para generar representaciones vectoriales con Skip-gram
x_features_skip2 = df1_clean['Item'].apply(lambda x: build_fasttext_embedding(x, model_skip))

In [ ]:
nan_count = sum(1 for x in x_features_skip2 if isinstance(x, float) and np.isnan(x))
print(f"Número de valores NaN: {nan_count}")

Número de valores NaN: 0


In [ ]:
x_features_skip2

,Item
0,"[-0.22827701, 0.32150352, -0.2419231, -0.15237..."
1,"[-0.30345526, 0.48847383, -0.10112117, -0.1176..."
2,"[-0.1672898, 0.45484948, -0.22749312, -0.10959..."
3,"[-0.23652862, 0.4432241, -0.21453227, -0.16783..."
4,"[-0.3359639, 0.4533804, -0.14189483, -0.103106..."
...,...
84597,"[-0.22127761, -0.031675063, -0.011125708, 0.14..."
84598,"[-0.07723925, 0.007497979, 0.019145897, 0.1509..."
84599,"[-0.3275132, 0.1306753, -0.07227141, 0.0520096..."
84600,"[-0.3275132, 0.1306753, -0.07227141, 0.0520096..."


In [ ]:
x_features_skip_final = pd.DataFrame(x_features_skip2.tolist())
x_features_skip_final

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,-0.228277,0.321504,-0.241923,-0.152376,0.304912,-0.242363,0.614902,0.255421,0.260346,0.264693,...,0.405087,0.146415,0.538939,-0.257472,0.137234,-0.235385,0.138139,0.406731,0.023569,0.011637
1,-0.303455,0.488474,-0.101121,-0.117618,0.232587,-0.185490,0.225194,0.109712,0.126604,0.401582,...,0.236617,0.115444,0.561656,-0.248810,-0.005176,-0.333617,-0.209544,0.209303,-0.162815,0.147726
2,-0.167290,0.454849,-0.227493,-0.109593,0.326680,-0.206344,0.420211,0.212928,0.239036,0.363807,...,0.414301,0.101980,0.571308,-0.206124,0.037776,-0.306458,0.012630,0.374803,-0.033967,0.083662
3,-0.236529,0.443224,-0.214532,-0.167831,0.226845,-0.158724,0.327293,0.118164,0.209829,0.395172,...,0.317987,0.060547,0.624980,-0.210960,-0.010315,-0.314601,-0.083370,0.353112,-0.136673,0.106068
4,-0.335964,0.453380,-0.141895,-0.103107,0.283859,-0.239794,0.436948,0.180671,0.111600,0.375315,...,0.317476,0.232029,0.362996,-0.353336,0.125858,-0.277107,0.007130,0.248913,-0.070651,0.001575
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84597,-0.221278,-0.031675,-0.011126,0.141354,-0.002474,0.356283,0.299558,-0.153400,-0.001130,0.236455,...,0.504408,-0.131574,0.199112,-0.084996,0.300493,-0.309252,0.023567,0.169134,-0.086932,0.002475
84598,-0.077239,0.007498,0.019146,0.150984,0.193751,0.251958,0.585314,-0.254827,-0.045652,0.333128,...,-0.082281,0.005406,0.726584,0.120416,0.296818,-0.525939,-0.482437,0.279715,-0.053542,-0.369728
84599,-0.327513,0.130675,-0.072271,0.052010,0.152370,0.292574,0.330325,-0.119689,0.025331,0.351199,...,0.595346,0.057403,-0.001371,-0.296158,0.461527,-0.090893,0.000451,0.087484,-0.029723,-0.086651
84600,-0.327513,0.130675,-0.072271,0.052010,0.152370,0.292574,0.330325,-0.119689,0.025331,0.351199,...,0.595346,0.057403,-0.001371,-0.296158,0.461527,-0.090893,0.000451,0.087484,-0.029723,-0.086651


In [ ]:
# Predicciones para Skip-gram
pred_skip_final = best_model_skip.predict(x_features_skip_final)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(


In [ ]:
pred_skip_final

array([321010303, 321030303, 321010303, ..., 312020137, 312020137,
       321030303])

In [ ]:
# Convertir a DataFrame
df_cod_1 = pd.DataFrame(pred_skip_final, columns=["codigo"])  # Puedes ajustar los nombres de las columnas
print(df_cod_1)

          codigo
0      321010303
1      321030303
2      321010303
3      321010303
4      321030303
...          ...
84597  312020137
84598          0
84599  312020137
84600  312020137
84601  321030303

[84602 rows x 1 columns]


In [ ]:
# Unir los DataFrames por columnas (horizontalmente)
df_combined = pd.concat([df1_clean, df_cod_1], axis=1)
df_combined

,Item,Marca,Precio,Categoria,Web,Fecha,genero,codigo
0,zapatillas urbanas adidas para hombre,adidas,160.30,zapatillas,oeschle,26-11-2024,hombre,321010303
1,zapatillas urbanas reebok mujer reebok jogger ...,reebok,129.00,zapatillas,oeschle,26-11-2024,mujer,321030303
2,zapatillas urbanas reebok para hombre jogg lite,reebok,182.78,zapatillas,oeschle,26-11-2024,hombre,321010303
3,zapatillas urbanas reebok hombre jogger lite,reebok,229.00,zapatillas,oeschle,26-11-2024,hombre,321010303
4,zapatillas urbanas para mujer adidas court pla...,adidas,249.00,zapatillas,oeschle,26-11-2024,mujer,321030303
...,...,...,...,...,...,...,...,...
84597,pantalón straight algodón mujer,stefano cocci,129.90,moda mujer,falabella,26-11-2024,mujer,312020137
84598,conjunto hombre polera más joggers entero hech...,generico,139.90,moda hombre,falabella,26-11-2024,hombre,0
84599,pantalón casual mujer,springfield,99.96,moda mujer,falabella,26-11-2024,mujer,312020137
84600,pantalón casual mujer,springfield,124.95,moda mujer,falabella,26-11-2024,mujer,312020137


In [ ]:
# Guardar las predicciones en un archivo CSV
pd.DataFrame(df_combined).to_csv('predicciones_finales_26_11.csv',encoding="utf-8", index=False)

## Prediciendo los códigos de la data extraida el 05/12

In [ ]:
# Data final 05/12
df2 = pd.read_csv("/content/drive/MyDrive/1. Clasificacion/Muestras/5_12_2024_genero.csv", encoding="utf-8")
df2.head()

,Item,Marca,Precio,Categoria,Web,Fecha,genero
0,zapatillas urbanas adidas para hombre,adidas,160.3,zapatillas,oeschle,12/05/2024,hombre
1,zapatillas urbanas reebok mujer reebok jogger ...,reebok,129.0,zapatillas,oeschle,12/05/2024,mujer
2,zapatillas urbanas reebok para hombre jogg lite,reebok,183.2,zapatillas,oeschle,12/05/2024,hombre
3,zapatillas urbanas reebok hombre jogger lite,reebok,229.0,zapatillas,oeschle,12/05/2024,hombre
4,zapatillas urbanas para mujer adidas court pl...,adidas,249.0,zapatillas,oeschle,12/05/2024,mujer


In [ ]:
df2 = df2.dropna(subset=['Precio'])
df2.isnull().sum()

,0
Item,0
Marca,0
Precio,0
Categoria,0
Web,0
Fecha,0
genero,0


### Pre - Procesamiento

In [ ]:
#Conviertiendo a minúsculas
df2["Item"]=df2["Item"].str.lower()

#Eliminando espacios en blaco al comienzo y al final
df2["Item"]=df2["Item"].str.strip()

#Eliminando ": ; ,"
df2["Item"]=df2["Item"].str.replace(":","")
df2["Item"]=df2["Item"].str.replace(";","")
df2["Item"]=df2["Item"].str.replace(",","")

#Eliminar espacios redundantes entre palabras
df2["Item"] = df2["Item"].str.replace(r'\s+', ' ', regex=True)

df2.head()

,Item,Marca,Precio,Categoria,Web,Fecha,genero
0,zapatillas urbanas adidas para hombre,adidas,160.3,zapatillas,oeschle,12/05/2024,hombre
1,zapatillas urbanas reebok mujer reebok jogger ...,reebok,129.0,zapatillas,oeschle,12/05/2024,mujer
2,zapatillas urbanas reebok para hombre jogg lite,reebok,183.2,zapatillas,oeschle,12/05/2024,hombre
3,zapatillas urbanas reebok hombre jogger lite,reebok,229.0,zapatillas,oeschle,12/05/2024,hombre
4,zapatillas urbanas para mujer adidas court pla...,adidas,249.0,zapatillas,oeschle,12/05/2024,mujer


In [ ]:
# Aplicar la función para generar representaciones vectoriales con Skip-gram
x_features_skip = df2['Item'].apply(lambda x: build_fasttext_embedding(x, model_skip))

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [ ]:
nan_count = sum(1 for x in x_features_skip if isinstance(x, float) and np.isnan(x))
print(f"Número de valores NaN: {nan_count}")

Número de valores NaN: 49


In [ ]:
# Identificar los índices donde hay NaN
nan_indices = [i for i, feature in enumerate(x_features_skip) if isinstance(feature, float) and np.isnan(feature)]

print(f"Índices con NaN: {nan_indices}")

Índices con NaN: [7458, 30523, 30525, 40240, 40314, 40346, 40696, 40697, 40755, 40780, 40897, 42043, 42288, 42564, 45214, 45388, 46537, 47427, 48260, 48484, 49184, 50100, 50101, 50924, 57098, 57099, 57100, 58117, 58124, 60736, 61521, 62368, 62387, 62436, 62437, 62476, 66620, 68227, 72482, 73666, 73683, 74699, 80099, 80811, 86588, 88504, 88697, 88792, 88950]


In [ ]:
# Eliminar las filas de df2 con esos índices
df2_clean = df2.drop(index=nan_indices)
df2_clean = df2_clean.reset_index(drop=True)
df2_clean

,Item,Marca,Precio,Categoria,Web,Fecha,genero
0,zapatillas urbanas adidas para hombre,adidas,160.30,zapatillas,oeschle,12/05/2024,hombre
1,zapatillas urbanas reebok mujer reebok jogger ...,reebok,129.00,zapatillas,oeschle,12/05/2024,mujer
2,zapatillas urbanas reebok para hombre jogg lite,reebok,183.20,zapatillas,oeschle,12/05/2024,hombre
3,zapatillas urbanas reebok hombre jogger lite,reebok,229.00,zapatillas,oeschle,12/05/2024,hombre
4,zapatillas urbanas para mujer adidas court pla...,adidas,249.00,zapatillas,oeschle,12/05/2024,mujer
...,...,...,...,...,...,...,...
91974,conjunto hombre polera más joggers entero hech...,generico,139.90,moda hombre,falabella,12/05/2024,hombre
91975,pantalón casual mujer,springfield,99.96,moda mujer,falabella,12/05/2024,mujer
91976,pantalón casual mujer,springfield,124.95,moda mujer,falabella,12/05/2024,mujer
91977,zapatilla para dama kenia,bashira,99.98,zapatillas urbanas mujer,falabella,12/05/2024,mujer


In [ ]:
# Aplicar la función para generar representaciones vectoriales con Skip-gram
x_features_skip2 = df2_clean['Item'].apply(lambda x: build_fasttext_embedding(x, model_skip))

In [ ]:
nan_count = sum(1 for x in x_features_skip2 if isinstance(x, float) and np.isnan(x))
print(f"Número de valores NaN: {nan_count}")

Número de valores NaN: 0


In [ ]:
x_features_skip2

,Item
0,"[-0.22827701, 0.32150352, -0.2419231, -0.15237..."
1,"[-0.30345526, 0.48847383, -0.10112117, -0.1176..."
2,"[-0.1672898, 0.45484948, -0.22749312, -0.10959..."
3,"[-0.23652862, 0.4432241, -0.21453227, -0.16783..."
4,"[-0.3359639, 0.4533804, -0.14189483, -0.103106..."
...,...
91974,"[-0.07723925, 0.007497979, 0.019145897, 0.1509..."
91975,"[-0.3275132, 0.1306753, -0.07227141, 0.0520096..."
91976,"[-0.3275132, 0.1306753, -0.07227141, 0.0520096..."
91977,"[-0.21610416, 0.25084272, -0.06320541, -0.1141..."


In [ ]:
x_features_skip_final = pd.DataFrame(x_features_skip2.tolist())
x_features_skip_final

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,-0.228277,0.321504,-0.241923,-0.152376,0.304912,-0.242363,0.614902,0.255421,0.260346,0.264693,...,0.405087,0.146415,0.538939,-0.257472,0.137234,-0.235385,0.138139,0.406731,0.023569,0.011637
1,-0.303455,0.488474,-0.101121,-0.117618,0.232587,-0.185490,0.225194,0.109712,0.126604,0.401582,...,0.236617,0.115444,0.561656,-0.248810,-0.005176,-0.333617,-0.209544,0.209303,-0.162815,0.147726
2,-0.167290,0.454849,-0.227493,-0.109593,0.326680,-0.206344,0.420211,0.212928,0.239036,0.363807,...,0.414301,0.101980,0.571308,-0.206124,0.037776,-0.306458,0.012630,0.374803,-0.033967,0.083662
3,-0.236529,0.443224,-0.214532,-0.167831,0.226845,-0.158724,0.327293,0.118164,0.209829,0.395172,...,0.317987,0.060547,0.624980,-0.210960,-0.010315,-0.314601,-0.083370,0.353112,-0.136673,0.106068
4,-0.335964,0.453380,-0.141895,-0.103107,0.283859,-0.239794,0.436948,0.180671,0.111600,0.375315,...,0.317476,0.232029,0.362996,-0.353336,0.125858,-0.277107,0.007130,0.248913,-0.070651,0.001575
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91974,-0.077239,0.007498,0.019146,0.150984,0.193751,0.251958,0.585314,-0.254827,-0.045652,0.333128,...,-0.082281,0.005406,0.726584,0.120416,0.296818,-0.525939,-0.482437,0.279715,-0.053542,-0.369728
91975,-0.327513,0.130675,-0.072271,0.052010,0.152370,0.292574,0.330325,-0.119689,0.025331,0.351199,...,0.595346,0.057403,-0.001371,-0.296158,0.461527,-0.090893,0.000451,0.087484,-0.029723,-0.086651
91976,-0.327513,0.130675,-0.072271,0.052010,0.152370,0.292574,0.330325,-0.119689,0.025331,0.351199,...,0.595346,0.057403,-0.001371,-0.296158,0.461527,-0.090893,0.000451,0.087484,-0.029723,-0.086651
91977,-0.216104,0.250843,-0.063205,-0.114137,0.356236,-0.101318,0.443638,0.163986,0.147345,0.228028,...,0.451824,0.239342,0.305887,-0.297879,0.403428,-0.217046,0.003460,0.348926,0.180955,-0.261289


In [ ]:
# Predicciones para Skip-gram
pred_skip_final = best_model_skip.predict(x_features_skip_final)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(


In [ ]:
pred_skip_final

array([321010303, 321030303, 321010303, ..., 312020137, 321030303,
       321030303])

In [ ]:
# Convertir a DataFrame
df_cod = pd.DataFrame(pred_skip_final, columns=["codigo"])  # Puedes ajustar los nombres de las columnas
print(df_cod)

          codigo
0      321010303
1      321030303
2      321010303
3      321010303
4      321030303
...          ...
91974          0
91975  312020137
91976  312020137
91977  321030303
91978  321030303

[91979 rows x 1 columns]


In [ ]:
# Unir los DataFrames por columnas (horizontalmente)
df_combined = pd.concat([df2_clean, df_cod], axis=1)
df_combined

,Item,Marca,Precio,Categoria,Web,Fecha,genero,codigo
0,zapatillas urbanas adidas para hombre,adidas,160.30,zapatillas,oeschle,12/05/2024,hombre,321010303
1,zapatillas urbanas reebok mujer reebok jogger ...,reebok,129.00,zapatillas,oeschle,12/05/2024,mujer,321030303
2,zapatillas urbanas reebok para hombre jogg lite,reebok,183.20,zapatillas,oeschle,12/05/2024,hombre,321010303
3,zapatillas urbanas reebok hombre jogger lite,reebok,229.00,zapatillas,oeschle,12/05/2024,hombre,321010303
4,zapatillas urbanas para mujer adidas court pla...,adidas,249.00,zapatillas,oeschle,12/05/2024,mujer,321030303
...,...,...,...,...,...,...,...,...
91974,conjunto hombre polera más joggers entero hech...,generico,139.90,moda hombre,falabella,12/05/2024,hombre,0
91975,pantalón casual mujer,springfield,99.96,moda mujer,falabella,12/05/2024,mujer,312020137
91976,pantalón casual mujer,springfield,124.95,moda mujer,falabella,12/05/2024,mujer,312020137
91977,zapatilla para dama kenia,bashira,99.98,zapatillas urbanas mujer,falabella,12/05/2024,mujer,321030303


In [ ]:
# Guardar las predicciones en un archivo CSV
pd.DataFrame(df_combined).to_csv('predicciones_finales_5_12.csv', encoding="utf-8", index=False)

# Asignando la clase en función al código predicho

In [ ]:
diccionario=pd.read_csv("/content/drive/MyDrive/1. Clasificacion/Muestras/diccionario.csv", encoding="utf-8")
diccionario.head()

,codigo,class_inei
0,312010119,CALCETINES (HOMBRE > 14)
1,312010124,CAMISA (HOMBRE > 14)
2,312010132,"CASACA, CHAQUETA, GUERRERA (HOMBRE > 14)"
3,312010138,CHOMPA / SUÉTER (HOMBRE > 14)
4,312010153,PANTALÓN CORTO (HOMBRE > 14)


In [ ]:
data_5_12=pd.read_csv("/content/drive/MyDrive/1. Clasificacion/Muestras/predicciones_finales_5_12.csv", encoding="utf-8")
data_26_11=pd.read_csv("/content/drive/MyDrive/1. Clasificacion/Muestras/predicciones_finales_26_11.csv", encoding="utf-8")

In [ ]:
data_5_12_merged=pd.merge(data_5_12, diccionario, on="codigo", how="left")
data_5_12_merged.head()

,Item,Marca,Precio,Categoria,Web,Fecha,genero,codigo,class_inei
0,zapatillas urbanas adidas para hombre,adidas,160.3,zapatillas,oeschle,12/05/2024,hombre,321010303,ZAPATILLAS (HOMBRE >14)
1,zapatillas urbanas reebok mujer reebok jogger ...,reebok,129.0,zapatillas,oeschle,12/05/2024,mujer,321030303,ZAPATILLAS (MUJER > 14)
2,zapatillas urbanas reebok para hombre jogg lite,reebok,183.2,zapatillas,oeschle,12/05/2024,hombre,321010303,ZAPATILLAS (HOMBRE >14)
3,zapatillas urbanas reebok hombre jogger lite,reebok,229.0,zapatillas,oeschle,12/05/2024,hombre,321010303,ZAPATILLAS (HOMBRE >14)
4,zapatillas urbanas para mujer adidas court pla...,adidas,249.0,zapatillas,oeschle,12/05/2024,mujer,321030303,ZAPATILLAS (MUJER > 14)


In [ ]:
data_26_11_merged=pd.merge(data_26_11, diccionario, on="codigo", how="left")
data_26_11_merged.head()

,Item,Marca,Precio,Categoria,Web,Fecha,genero,codigo,class_inei
0,zapatillas urbanas adidas para hombre,adidas,160.30,zapatillas,oeschle,26-11-2024,hombre,321010303,ZAPATILLAS (HOMBRE >14)
1,zapatillas urbanas reebok mujer reebok jogger ...,reebok,129.00,zapatillas,oeschle,26-11-2024,mujer,321030303,ZAPATILLAS (MUJER > 14)
2,zapatillas urbanas reebok para hombre jogg lite,reebok,182.78,zapatillas,oeschle,26-11-2024,hombre,321010303,ZAPATILLAS (HOMBRE >14)
3,zapatillas urbanas reebok hombre jogger lite,reebok,229.00,zapatillas,oeschle,26-11-2024,hombre,321010303,ZAPATILLAS (HOMBRE >14)
4,zapatillas urbanas para mujer adidas court pla...,adidas,249.00,zapatillas,oeschle,26-11-2024,mujer,321030303,ZAPATILLAS (MUJER > 14)


### Exportando los datos con los que se calculará el IPC

In [ ]:
data_5_12_merged.to_csv('datos_5_12.csv', encoding="utf-8", index=False)
data_26_11_merged.to_csv('datos_26_11.csv', encoding="utf-8", index=False)